In [12]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from datasets import Dataset, ClassLabel
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer

In [13]:
# train_balanced.py
# Treina modelo DistilBERT com oversampled data

import pandas as pd
import numpy as np
import torch
from sklearn.metrics import f1_score
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset, ClassLabel
from sklearn.preprocessing import LabelEncoder

# 1. Carregar dataset balanceado
print("📄 Lendo dados balanceados...")
df = pd.read_csv(r"C:\Users\Ana\Desktop\project_ENHESA\data\final_data.csv")


📄 Lendo dados balanceados...


In [14]:
# 2. Codificar labels (se necessário)
print("🔠 Codificando labels...")
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["target"])
labels = label_encoder.classes_.tolist()

# 3. Tokenizador
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_function(batch):
    return tokenizer(batch["synopsis"], padding=True, truncation=True)

# 4. Preparar dataset Hugging Face
hf_dataset = Dataset.from_pandas(df[["synopsis", "label"]])
hf_dataset = hf_dataset.cast_column("label", ClassLabel(num_classes=len(labels), names=labels))
hf_dataset = hf_dataset.train_test_split(test_size=0.2, stratify_by_column="label", seed=42)
hf_dataset = hf_dataset.map(tokenize_function, batched=True)

# 5. Modelo
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(labels))

# 6. Métrica de avaliação
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, preds, average="macro")
    return {"f1_macro": f1}

# 7. Argumentos de treino
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_steps=10
)

# 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 9. Treinar
print("🚀 Iniciando treino...")
trainer.train()

# 10. Avaliar
print("📊 Avaliação:")
metrics = trainer.evaluate()
print(metrics)

# 11. Salvar modelo
print("💾 Salvando modelo em 'modelo_distilbert'...")
trainer.save_model("modelo_distilbert")
tokenizer.save_pretrained("modelo_distilbert")

🔠 Codificando labels...



Casting the dataset: 100%|██████████| 3075/3075 [00:00<00:00, 30090.16 examples/s]




Map: 100%|██████████| 2460/2460 [00:02<00:00, 955.05 examples/s]


Map: 100%|██████████| 615/615 [00:00<00:00, 986.16 examples/s] 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Ana\Desktop\project_ENHESA\.venv\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/232 [09:43<?, ?it/s]


🚀 Iniciando treino...


  0%|          | 0/924 [00:00<?, ?it/s]c:\Users\Ana\Desktop\project_ENHESA\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
  1%|          | 10/924 [01:52<2:53:03, 11.36s/it]

{'loss': 1.0924, 'grad_norm': 3.170321464538574, 'learning_rate': 1.9783549783549785e-05, 'epoch': 0.03}


...

{'loss': 0.1691, 'grad_norm': 3.2920584678649902, 'learning_rate': 7.012987012987014e-06, 'epoch': 1.95}


 66%|██████▌   | 610/924 [1:50:49<55:15, 10.56s/it]

{'loss': 0.155, 'grad_norm': 8.252318382263184, 'learning_rate': 6.796536796536797e-06, 'epoch': 1.98}


 67%|██████▋   | 616/924 [1:51:45<45:37,  8.89s/it]
















































KeyboardInterrupt: 